<a href="https://colab.research.google.com/github/nouman-tariq/python_projects/blob/main/tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Computer Vision - lab 7 
### Read, Write and Display a video using OpenCV

In [ ]:
import numpy as np
import cv2 as cv 
import os
import glob
import matplotlib.pyplot as plt
from numpy.random import uniform

### Reading and displaying a video

In [ ]:
base_folder = "data"
path_video1 = os.path.join(base_folder, "3.mp4")

# Open the video 
cap = cv.VideoCapture(path_video1) 

if cap.isOpened() == False: 
    print("Error opening video stream or file") 
    
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
print('frame_width = ' + str(frame_width))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
print('frame_height = ' + str(frame_height))

length = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
print("total number of frames = " + str(length))

fps = int(cap.get(cv.CAP_PROP_FPS))
print("frames per second = " + str(fps))

cap.release()

frame_width = 1280
frame_height = 720
total number of frames = 196
frames per second = 30


In [ ]:
# Display the video
def display_video(video_path: str, max_number_of_frame_to_run: int = None) -> None:
    """
    This function display the video of the screen frame by frame.
    :param video_path: Path to the video
    :param max_number_of_frame_to_run: Set how many frame to be displayed. If None all frames will be displayed.
    """
    
    current_frame = 0 
    
    # Open the video 
    cap = cv.VideoCapture(path_video1)

    while cap.isOpened(): 

        ret, frame = cap.read() # Read the frame 
        if ret is True:
            current_frame = current_frame + 1 
            cv.imshow("Frame", frame)

            if max_number_of_frame_to_run is not None and current_frame > max_number_of_frame_to_run:
                break

            if cv.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    # after playing the video, release the video capture    
    cap.release()
    # close all the frames
    cv.destroyAllWindows()

In [ ]:
display_video(video_path=os.path.join(base_folder, "3.mp4"), max_number_of_frame_to_run=750)

### Writing a video

In [ ]:
def read_frames(video_path):
    """
    This function takes the video path and returns the a list of frames.
    :param video_path: Path to the video
    """
    frames = []
    cap = cv.VideoCapture(video_path)  
    if cap.isOpened() == False: 
        raise Exception("Error opening video stream or file") 
        return frames
    
    while cap.isOpened():  
        ret, frame = cap.read() # Read the frame
        if ret is True:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames

In [ ]:
# We are going to write the same video at 1 fps, first we need to read the frames.
frames = read_frames(os.path.join(base_folder, "3.mp4"))

# here we have the extensions and the fourcc for each of it
video_extension_and_fourcc_dict = {'avi': cv.VideoWriter_fourcc('M', 'J', 'P', 'G'),
                                   'mp4': 0x7634706d}   
 
# We need to create a VideoWriter object. 
# First, we should specify the output file name with its format (eg: 1_fps_1.mp4). 
# We should specify the FourCC code and the number of frames per second (FPS). 
# Lastly, the frame size should be passed (width, height).

video_output_name = "3_fps_1.mp4"
output_video = cv.VideoWriter(video_output_name, video_extension_and_fourcc_dict["mp4"], 1,
                              (frames[0].shape[1], frames[0].shape[0]))

num_frames = len(frames)
# We know that the first video has 30 fps.
for i in range(0, num_frames, 30):
    output_video.write(frames[i]) # writing the frame

# don't forget to release the video writer
output_video.release()

# Video analysis of a snooker footage

Goal was to develop an automatic system for video analysis of snooker footages. The system should be able to detect the snooker table and the balls on the table, track the balls, infer when a ball is potted into a pocket. We'll focus about tracking the cue ball (white ball) and another ball. The initial bounding boxes of the two balls are provided for the first frame (they follow the format [xmin ymin xmax ymax], where (xmin, ymin) is the top left corner and (xmax, ymax) is the bottom right corner of the initial bounding-box). In a video, we consider that the algorithm correctly tracks a ball if in more (greater or equal) than 80% of the frames the algorithm correctly localizes the ball to be tracked. We consider that the algorithm correctly localizes the ball to be tracked in a specific frame if the value of the IOU (intersection over union) between the window provided by your algorithm and the ground-truth window is more than 20%. 

## Tracking using template matching

In [ ]:
def select_roi(frame):
    """
    Select the roi from the image.
    :param frame
    :return roi, x, y, w, h
    """
    x, y, w, h = cv.selectROI(frame) 
    track_window = (x, y, w, h) 
    roi = frame[y: y + h, x: x + w]
    
    annotated_frame = cv.rectangle(frame, (x, y), (x + w, y + h), 255, 2) 
    cv.imshow('First frame initialization', annotated_frame)
    cv.waitKey(2000)
    cv.destroyAllWindows()
         
    return roi, x, y, w, h

def find_ball_using_template_matching(frame, roi_gray, old_bbox):
    """
    :param frame: Current frame
    :param roi_gray: the previous detected region
    :param old_bbox: the previous detected bbox corresponding to roi_gray [x, y, w, h].
    :return new_x, new_y
    """
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    x, y, w, h = old_bbox
    

    # define the searching region for template matching
    # a rectangular region 4h * 4w
    center = (y + h//2, x + h//2)
    y_min = np.max((0, center[0] - (2*h)))
    y_max = np.min((frame.shape[0], center[0] + (2*h)))
    x_min = np.max((0, center[1] - (2*w)))
    x_max = np.min((frame.shape[1], center[1] + (2*w)))
    
    # display the searching region
    mask1 = np.int8(np.zeros(frame_gray.shape))
    mask1[y_min: y_max, x_min: x_max] = 255 
    frame_gray_mask = cv.bitwise_and(frame_gray,frame_gray,mask=mask1)
    cv.imshow('frame gray mask', frame_gray_mask)
    cv.waitKey(500)

    # function cv.matchTemplate works only on grayscale templates
    res = cv.matchTemplate(frame_gray_mask, roi_gray, cv.TM_CCOEFF_NORMED)        
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)

    new_y = max_loc[1]
    new_x = max_loc[0]
    return new_x, new_y
    
def track_ball_using_template_matching(video_path) -> list:
    """
    This function track the ball (which is initialized using select ROI) using template matching:
    Template matching: https://docs.opencv.org/master/df/dfb/group__imgproc__object.html#ga586ebfb0a7fb604b35a23d85391329be
    :param video_path: The path to the video.
    :return bounding boxes where the ball was found.
    """
    
    bboxes = []
    
    cap = cv.VideoCapture(video_path)
    ret, first_frame = cap.read() # Read the first frame 
    
    roi, x, y, w, h = select_roi(first_frame)  
    roi_gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
    frame_idx = 0
    
    while cap.isOpened():
        frame_idx += 1
        ret, frame = cap.read()

        if ret is True: 
            x, y = find_ball_using_template_matching(frame, roi_gray, old_bbox=[x, y, w, h])
            bboxes.append([frame_idx, x, y, x + w, y + h])
            
            # show annotated image with new bbox
            annotated_image = cv.rectangle(frame, (x, y), (x + w, y + h), 255, 2)
            cv.imshow('annotated_image', annotated_image)

            if cv.waitKey(25) & 0xFF == ord('q'):
                break
                
        else:
            break  
            
    cap.release() 
    cv.destroyAllWindows()
    return bboxes

In [ ]:
base_folder = "data"
# 3, 4, 8, 9, 11, 19, 20
path_video1 = os.path.join(base_folder, "3.mp4")
bboxes = track_ball_using_template_matching(path_video1)

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

def compute_percentage_tracking(gt_bboxes, predicted_bboxes, num_frames):
    """
    This function compute the percentage of detected bounding boxes based on the ground-truth bboxes and the predicted ones.
    :param gt_bboxes. The ground-truth bboxes with the format: frame_idx, x_min, y_min, x_max, y_max.
    :param predicted_bboxes. The predicted bboxes with the format: frame_idx, x_min, y_min, x_max, y_max
    :param num_frames. The total number of frames in the video.
    """
    
    num_frames = int(num_frames)
    
    tp = 0
    fp = 0
    
    gt_dict = {}
    for gt_box in gt_bboxes:
        gt_dict[gt_box[0]] = gt_box[1:]
    
    pred_dict = {}
    for pred_bbox in predicted_bboxes:
        pred_dict[pred_bbox[0]] = pred_bbox[1:]
        
    for i in range(num_frames):
        if gt_dict.get(i, None) is None and pred_dict.get(i, None) is None: # the ball is not on the table
            tp += 1 
        
        elif gt_dict.get(i, None) is not None and pred_dict.get(i, None) is None: # the ball is not detected
            fp += 1
            
        elif gt_dict.get(i, None) is None and pred_dict.get(i, None) is not None: # the ball is not on the table, but it is 'detected'
            fp += 1
            
        elif gt_dict.get(i, None) is not None and pred_dict.get(i, None) is not None: # the ball is on the table and it is detected
            
            iou = bb_intersection_over_union(gt_dict[i], pred_dict[i])
            if iou >= 0.2:
                tp += 1
            else:
                fp += 1 
             
            
    print(f'tp = {tp}, fp = {fp}')
    assert tp + fp == num_frames
    perc = tp / (tp + fp)
    
    return perc

In [ ]:
# load the ground-truth file
black_ball_gt = np.loadtxt(os.path.join(base_folder, '3_ball_2_gt.txt'))
# the first line contains the lenght (number of frames) of the video (followed by -1 in order to keep the dimension of the array)
print(black_ball_gt[0])

white_ball_gt = np.loadtxt(os.path.join(base_folder, '3_ball_1_gt.txt'))
print(white_ball_gt[0])

[196.  -1.  -1.  -1.  -1.]
[196.  -1.  -1.  -1.  -1.]


In [ ]:
compute_percentage_tracking(white_ball_gt[1:], bboxes, white_ball_gt[0][0])

tp = 190, fp = 6


0.9693877551020408

## Color histrogram for detecting the snooker balls

In [ ]:
def compute_color_histogram(img, bins_0, bins_1, bins_2):
    """
    Compute the color histogram of a window in the quantized BGR color space.
    :param img. 
    :param bins_0. The number of bins for the first channel.
    :param bins_1. The number of bins for the second channel.
    :param bins_2. The number of bins for the third channel.
    :return The histogram.
    """
    
    histogram = np.zeros((bins_0, bins_1, bins_2))
    # TODO: create the histogram by placing each b, g, r value to the corresponding bin.
        
    return histogram.flatten()
 
def compute_color_histogram_spatial_pyramid(img, bins_0, bins_1, bins_2):
    """
    Spatial pyramid with one level: the initial color histograsms + the color histograms of the 4 subimages.
    :param img. 
    :param bins_0. The number of bins for the first channel.
    :param bins_1. The number of bins for the second channel.
    :param bins_2. The number of bins for the third channel.
    :return The histogram.
    """
    
    histogram1 = compute_color_histogram(img, bins_0, bins_1, bins_2)
    
    h = img.shape[0]
    w = img.shape[1]
    
    histogram2 = compute_color_histogram(img[:h//2, :w//2], bins_0, bins_1, bins_2)
    histogram3 = compute_color_histogram(img[h//2: h, w//2: w], bins_0, bins_1, bins_2)
    histogram4 = compute_color_histogram(img[h//2: h, :w//2], bins_0, bins_1, bins_2)
    histogram5 = compute_color_histogram(img[h//2: h, w//2:w], bins_0, bins_1, bins_2)        
    
    histogram = np.array([histogram1, histogram2, histogram3, histogram4, histogram5])
    
    return histogram.flatten()


In [ ]:
# Select ROI
video_path = os.path.join(base_folder, "3.mp4")
cap = cv.VideoCapture(video_path)
ret, first_frame = cap.read() # Read the first frame 
cap.release()

img_crop, _, _, _, _ = select_roi(first_frame)


In [ ]:
# compute the color histogram of the window in the quantized BGR color space
# use our function
histogram_img = compute_color_histogram(img_crop, 4, 4, 4)
print('Color histogram in the quantized BGR: 4, 4, 4')
print(histogram_img.shape)
print(histogram_img.sum())
print(img_crop.shape[0] * img_crop.shape[1])
print(histogram_img)

print('Color histogram in with special pyramid: 4, 4, 4')
histogram_img = compute_color_histogram_spatial_pyramid(img_crop, 4, 4, 4)
print(histogram_img.shape)
print(histogram_img.sum())


# compute the color histogram of the window in the quantized BGR color space
# use the function provided by OpenCV
print('Color histogram (from OpenCV) in the quantized BGR: 4, 4, 4')
hist_img = cv.calcHist([img_crop], [0, 1, 2], None, [4, 4, 4], [0, 256, 0, 256, 0, 256]) 
print(hist_img.flatten())

Color histogram in the quantized BGR: 4, 4, 4
(64,)
1020.0
1020
[ 44.   0.   0.   0.  70.  38.   0.   0. 333.  58.   4.   0.   1.   2.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   4.  30.   0.
   0.  11.  38.   8.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.  19. 177. 183.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.]
Color histogram in with special pyramid: 4, 4, 4
(320,)
2040.0
Color histogram (from OpenCV) in the quantized BGR: 4, 4, 4
[ 44.   0.   0.   0.  70.  38.   0.   0. 333.  58.   4.   0.   1.   2.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   4.  30.   0.
   0.  11.  38.   8.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.  19. 177. 183.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.]


## Tracking using particle filtering

In [ ]:
def create_uniform_particles(x_range, y_range, N):
    """
    A particle is a bounding box, represented by the top left corner and fixed width and height
    """
    particles = np.empty((N, 2))
    particles[:, 0] = uniform(x_range[0], x_range[1], size=N)
    particles[:, 1] = uniform(y_range[0], y_range[1], size=N) 
    return particles


def predict(particles, velocity, std, frame, w, h):
    """
    Predict where the particles will be at the next frame by applying some dynamics
    take into account velocity and some random noise.
    """
    
    N = len(particles)    
    
    noise = np.random.randn(N) * std[0]  
    for i in range(N):
        particles[i, 0] = particles[i, 0] + velocity[0] + noise[i]
        # check that the particle is not outside of the image
        if(particles[i, 0] > frame.shape[1] - w):
            particles[i, 0] = frame.shape[1] - w
        if(particles[i, 0] < 0):
            particles[i, 0] = 0
            
    noise = np.random.randn(N) * std[1]
    for i in range(N):
        particles[i, 1] = particles[i, 1] + velocity[1] + noise[i]
        if(particles[i, 1] > frame.shape[0] - h):
            particles[i, 1] = frame.shape[0] - h
        if(particles[i, 1] < 0):
            particles[i, 1] = 0 
    return particles


def update(particles, frame, hist_roi_norm, w, h):
    """
    Update the weight of each particle based on how similar is to the target window 
    use a simple color histogram model essential step: how to update the weights.
    """
    particles = np.int32(particles)   
    weights = np.zeros((particles.shape[0]))
    for i in range(particles.shape[0]):
        img_particle = frame[particles[i, 1]: particles[i, 1] + h - 1, particles[i, 0]:particles[i, 0] + w - 1].copy()
        particle_hist = cv.calcHist([img_particle], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])  
        # do normalization
        particle_hist_norm = particle_hist / particle_hist.sum()        
      
        weights[i] = cv.compareHist(hist_roi_norm, particle_hist_norm, cv.HISTCMP_CHISQR_ALT)
 
        weights[i] = np.exp(-2 * (weights[i] ** 2))  
    weights += 1.e-10 # avoid round-off to zero
    # normalize the wights such that we have a probability distribution
    weights /= sum(weights) 
    return weights


def estimate(particles, weights):   
    """
    Estimate the center of the cloud of particles.
    """
    mean = np.float64(np.array([0, 0]))
    N = particles.shape[0]  
    for i in range(N): 
        mean += weights[i] * particles[i, :]   
    return mean


def resample(weights):
    """
    Resample particles based on their weight.
    """
    w = weights.flatten()
    N = len(w)    
    tries = np.random.multinomial(N, w)  
    indexes = np.zeros(N, 'i')
    cumsum_vector = np.cumsum(tries)
    pos = -1 
    for i in range(len(tries)):
        for j in range(tries[i]):            
            pos = pos + 1
            indexes[pos] = i
            
    return indexes

def resample_from_index(particles, weights, indexes):
    """
    Using the new indexes, keep only the particles and the weights corresponsing to the indexes. Then, re-normalize the weights.
    """
    particles[:] = particles[indexes]
    weights[:] = weights[indexes]
    weights /= np.sum(weights)
    return particles, weights

In [ ]:
def track_ball_using_particle_filtering(video_path: str, max_number_of_frame_to_run: int=None, num_particles=100):
    """
    Track the ball using particle filtering using the following steps:
    1. Generate the first particle using the first initialization of the ball.
    2. Predict the new locations of the particles using the velocity and some noise.
    3. Compute the weight of each particle based on their similarity with the first roi.
    4. Get the new location of the ball using a weighed sum of the particles.
    5. Resample particles based on their weight (the weight of a particle means the similarity with the first roi).
    
    :param video_path. Path to the video.
    :param max_number_of_frame_to_run. How many frames to track the ball.
    :param num_particles. How many particle to have.
    """
    cap = cv.VideoCapture(video_path)  
    current_frame = 0
    max_number_of_frame_to_run = 750

    ret, first_frame = cap.read() # Read the first frame         
    img_roi, x, y, w, h = select_roi(first_frame) 

    hist_roi = cv.calcHist([img_roi], [0 ,1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])  
    hist_roi_norm = hist_roi / hist_roi.sum() 
    bb = np.array([x, y, x + w, y + h])

    particles = ...
    velocity = [0, 0]
    std = [25, 25] 
    bboxes = []

    while(cap.isOpened()): 
        ret, frame = cap.read() # Read the frame   
        if ret is True: 
            current_frame = current_frame + 1   
            particles = ...     

            weights = ...

            obj = np.int32(...)       
            bboxes.append([current_frame, obj[0], obj[1], obj[0] + w, obj[1] + h])   

            velocity[0] = obj[0] - bb[0]
            velocity[1] = obj[1] - bb[1]      
            bb = obj.copy()

            indexes = ...
            particles, weights = ...          

            annotated_image = frame.copy()
            for i in range(num_particles):            
                annotated_image = cv.rectangle(annotated_image, (np.int32(particles[i, 0]), np.int32(particles[i, 1])), (np.int32(particles[i, 0]) + w, np.int32(particles[i, 1]) + h), (0, 255, 0), 1)

            annotated_image = cv.rectangle(annotated_image, (obj[0], obj[1]), (obj[0] + w, obj[1] + h), (0, 255, 255), 4)
            cv.imshow('annotated_image', annotated_image)   
            cv.waitKey(100)
            
            if max_number_of_frame_to_run is not None and current_frame > max_number_of_frame_to_run:
                break

            if cv.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    # after playing the video, release the video capture    
    cap.release()                
    # close all the frames
    cv.destroyAllWindows()
    return bboxes

In [ ]:
# change the path to match on your machine
base_folder = 'data'
video_path = os.path.join(base_folder, "20.mp4")
bboxes = track_ball_using_particle_filtering(video_path, max_number_of_frame_to_run=750, num_particles=100)

In [ ]:
# load the ground-truth file
red_ball_gt = np.loadtxt(os.path.join(base_folder, '20_ball_2_gt.txt'))  
white_ball_gt = np.loadtxt(os.path.join(base_folder, '20_ball_1_gt.txt')) 
compute_percentage_tracking(white_ball_gt[1:], bboxes, white_ball_gt[0][0])

tp = 235, fp = 1


0.9957627118644068

In [ ]:
cv.destroyAllWindows()